<a href="https://colab.research.google.com/github/karimghabbour/VirtualBuddy_RAG/blob/main/HPE_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1:Upload Documents to be Accessed (Mount to Drive)

In [2]:
#Mount Google Drive For Access
from google.colab import drive
drive.mount('/content/drive')

# Define the base path to the folder containing your PDFs
base_path = '/content/drive/My Drive/BasicRAG/'

# List of your PDF files
pdf_files = [
    'Resources inside HPE Barcelona Hub.pdf',
    'uncover-hidden-insights-create-business-advantage.pdf',
    'HPE GreenLake FAQ for technology innovators frequently asked questions-a00107144enw.pdf',
    'HPE Tech Care Service-a00108652enw.pdf'
]

# Full paths to the PDF files
pdf_paths = [base_path + pdf for pdf in pdf_files]



Mounted at /content/drive


#Step 2: Download Extra Packages

In [3]:
!pip install pymupdf pygpt4all sentence_transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 33.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.0/362.0 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.0/246.0 kB 31.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-

In [4]:
!pip install -U git+https://github.com/jerryjliu/llama_index.git
!pip install langchain


  Cloning https://github.com/jerryjliu/llama_index.git to /tmp/pip-req-build-kicn474n
  Running command git clone --filter=blob:none --quiet https://github.com/jerryjliu/llama_index.git /tmp/pip-req-build-kicn474n
  Resolved https://github.com/jerryjliu/llama_index.git to commit ce5963486d8b99fa045659dbdb7bd1398d6b7da5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 

#Step 3: Import Packages Required for Document Set up

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.legacy.embeddings.langchain import LangchainEmbedding
from llama_index.legacy import LLMPredictor

from llama_index.core import (
    GPTVectorStoreIndex,
    ServiceContext,
    StorageContext,
    download_loader
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Step 4: Load & Prepare Documents

In [6]:
PyMuPDFReader = download_loader("PyMuPDFReader")



<ipython-input-6-e65a49cf0750>:1: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PyMuPDFReader = download_loader("PyMuPDFReader")


In [7]:
documents = []
for path in pdf_paths:
    temp_docs = PyMuPDFReader().load(file_path=path, metadata=True)
    # Ensure document texts are not byte objects
    for doc in temp_docs:
        doc.text = doc.text.decode() if isinstance(doc.text, bytes) else doc.text
    documents.extend(temp_docs)

In [10]:
# testing docs
for i, pdf_path in enumerate(pdf_paths):
    # Prints the file name extracted from the path
    print(f"First page text from: {pdf_path.split('/')[-1]}")
    # Ensure there is at least one document to print from this PDF, and print the first 500 characters of the first document
    if len(documents) > i * len(documents) // len(pdf_paths):
        first_doc_index = i * len(documents) // len(pdf_paths)
        print(documents[first_doc_index].text[:500])  # Prints the first 500 characters to avoid too much output
    else:
        print("No documents available to print from this PDF.")
    print("\n---\n")


First page text from: Resources inside HPE Barcelona Hub.pdf
Resources inside HPE Barcelona Hub:  
 
- 
Issues with Laptops: Reach out to Karim Ghabbour 
- 
Issues with Wifi: Reach out to Johaness Hellwig 
- 
Info about NGQ and quoting: Reach out to Karim Hellwig  


---

First page text from: uncover-hidden-insights-create-business-advantage.pdf
Frequently asked questions 
Page 4 
 
TABLE 1. HPE GreenLake model top benefits and top differentiators 
HPE GreenLake top OEM benefits 
HPE GreenLake top differentiators 
Increased net new revenue from faster time to market 
True pay per use based on actual metered usage 
Eliminate overprovisioning and preserve cash flow 
Cloudlike experience securely on-premises 
Capacity available ahead of demand 
Preprovisioned buffer capacity to never run out 
Align IT spending to business goals 
Control ov

---

First page text from: HPE GreenLake FAQ for technology innovators frequently asked questions-a00107144enw.pdf
Page 6
Data sheet
Feature
Delivery

#Step 5: Set up Local Model & Embedding Model

In [ ]:
!pip install gpt4all

In [ ]:
# Update the path to the local LLM model stored in Google Drive
local_llm_path = '/content/drive/My Drive/BasicRAG/gpt4all-falcon-newbpe-q4_0.gguf'

# Initialize the model with the new path
llm = GPT4All(model=local_llm_path, backend='gptj', streaming=True, n_ctx=512)
llm_predictor = LLMPredictor(llm=llm)


In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))